# FMRIPREP BBX Preprocessing

---

In [18]:
# import pacakges 

import os, glob, shutil
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
import subprocess as sp

from openpyxl import load_workbook
from IPython.display import SVG, display
from datetime import date

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', 159)
#pd.set_option('display.max_colwidth', -1)
from matplotlib import rcParams
rcParams['axes.titlepad'] = 15

%matplotlib inline 

## Set Variables

In [19]:
# Variable load and setup
date = date.today()
study_folder_path="/projects/niblab/experiments/bbx"
data_folder_path= os.path.join(study_folder_path, "data")
report_folder_path=os.path.join(data_folder_path, "quality_analysis")
fmriprep_path='/projects/niblab/experiments/bbx/data/fmriprep'


In [9]:
fmriprep_folders=glob.glob(os.path.join(fmriprep_path, "sub-*/ses-*"))
fmriprep_ses1=[x.split("/")[-2] for x in fmriprep_folders if 'ses-1' in x]
fmriprep_ses2=[x.split("/")[-2] for x in fmriprep_folders if 'ses-2' in x]


s1_drop_subjects=[ "sub-029", "sub-047", "sub-049",  "sub-069", "sub-074", "sub-081", "sub-094", "sub-101", "sub-105",
               "sub-106", "sub-110", "sub-113", "sub-122", "sub-125", "sub-126", "sub-139", "sub-155",  
               "sub-158", "sub-165" ]


s2_drop_subjects=["sub-003", 'sub-012','sub-016', 'sub-018',  "sub-020", "sub-024", "sub-025",'sub-026', "sub-029",
                   'sub-035', "sub-047", "sub-049", "sub-052", 'sub-056', "sub-059", "sub-060", "sub-064",
                  "sub-069", "sub-074", "sub-076", "sub-077", "sub-081", 'sub-087', "sub-091", "sub-094", "sub-101", "sub-105",
                   "sub-106", 'sub-108', "sub-110", "sub-113", "sub-122", "sub-125", "sub-126", "sub-139", "sub-147", "sub-150",
                  "sub-155", "sub-158", "sub-165", "sub-167"]

fmriprep_ses1=[x for x in fmriprep_ses1 if x not in s1_drop_subjects]
fmriprep_ses2=[x for x in fmriprep_ses2 if x not in s2_drop_subjects]


runs=['run-1','run-2','run-3', 'run-4']
sessions=['ses-1','ses-2']
tasks=['task-training']
subject_list=list(set(fmriprep_ses1+fmriprep_ses2))

subject_list.sort()


#remove_subs=[ 'sub-029', 'sub-047', 'sub-049', 'sub-057', 'sub-069', 'sub-081', 'sub-105', 
 #             'sub-125', 'sub-126', 'sub-158', 'sub-165']


print(s2_drop_subjects)

['sub-003', 'sub-012', 'sub-016', 'sub-018', 'sub-020', 'sub-024', 'sub-025', 'sub-026', 'sub-029', 'sub-035', 'sub-047', 'sub-049', 'sub-052', 'sub-056', 'sub-059', 'sub-060', 'sub-064', 'sub-069', 'sub-074', 'sub-076', 'sub-077', 'sub-081', 'sub-087', 'sub-091', 'sub-094', 'sub-101', 'sub-105', 'sub-106', 'sub-108', 'sub-110', 'sub-113', 'sub-122', 'sub-125', 'sub-126', 'sub-139', 'sub-147', 'sub-150', 'sub-155', 'sub-158', 'sub-165', 'sub-167']


In [5]:
subject_list[:4]

['sub-001', 'sub-002', 'sub-003', 'sub-004']

In [6]:
"""
# Methods
"""

def quality_report():
    
    s1_bids_subject_list=[x.split("/")[-2] for x in 
                   glob.glob(os.path.join(study_folder_path, "data/bids/sub-*/ses-1"))]

    s1_bids_subject_list=[x for x in s1_bids_subject_list ]#if x not in remove_subs]

    s2_bids_subject_list=[x.split("/")[-2] for x in 
                       glob.glob(os.path.join(study_folder_path, "data/bids/sub-*/ses-2"))]

    s2_bids_subject_list=[x for x in s2_bids_subject_list]  #if x not in remove_subs]

    s1_fprep_subject_list=fmriprep_ses1
    s2_fprep_subject_list=fmriprep_ses2
    
    s1_fprep_missing = [x for x in s1_bids_subject_list if x not in s1_fprep_subject_list]
    s2_fprep_missing = [x for x in s2_bids_subject_list if x not in s2_fprep_subject_list]

    fprep_missing=list(set(s1_fprep_missing + s2_fprep_missing))
    fprep_missing.sort()

    print("[INFO] %s session 1 fmriprep subjects found. "%len(s1_fprep_subject_list))
    print("\n[INFO] %s session 1 bids subjects are not processed through fmriprep."%len(s1_fprep_missing))
    print("\n[INFO] %s \n\n"%s1_fprep_subject_list)
    print("[INFO] subjects: %s "%s1_fprep_missing)
    print("\n[INFO] %s session 2 fmriprep subjects found."%len(s2_fprep_subject_list))
    print("\n[INFO] %s session 2 bids subjects are not processed through fmriprep."%len(s2_fprep_missing))
    print("\n[INFO] subjects: %s "%s2_fprep_missing)
    print("\n[INFO] set of missing subjects: %s"%fprep_missing)
    print("\n[INFO] %s \n\n"%s2_fprep_subject_list)
    
    
    
    
"""
# Build FMRIPREP Report 

"""


def build_fmriprep_report(write_files=False):
    

    
    sessions=['ses-1', 'ses-2']
    dataframes=[]
        
    # loop through sessions
    for session in sessions:
        #print("\n[INFO] %s"%session)
        

        data_dict={} #initialize data dictionary for session

        # loop through subject set by subject
        for i in range(1,171):
            train_ct=0
            rest_ct=0
            if session == 'ses-2':
                rl_ct=0
            subject="sub-%s"%f'{i:03}'
            bids_folder=os.path.join(fmriprep_path, subject,
                                    session)

            if subject not in data_dict:
                data_dict[subject] = {}



            # get anat file and save plot
            #_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.g
            anat_img =os.path.join(fmriprep_path, subject, "anat",
                   '%s_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.g'%(subject))

            if os.path.exists(anat_img): 
                data_dict[subject]["anat"]="good"

            # get functional files and check their volume and plot images
            func_files=glob.glob(os.path.join(
                    fmriprep_path, subject, session,
                    "func/*task-*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" ))

            for func_file in func_files:
                
                task=func_file.split("/")[-1].split("_")[2]
                vol = sp.check_output(["fslnvols", func_file])
                vol=str(vol,'utf-8').strip("\n")


                if "resting" in task:
                    rest_ct+=1
                    var_name="%s_vol"%task
                    data_dict[subject][var_name]=vol
                elif "rl" in task:
                    rl_ct+=1
                    run=func_file.split("/")[-1].split("_")[3]
                    var_name="%s_%s_vol"%(task,run)
                    data_dict[subject][var_name]=vol
                elif "training" in task:
                    train_ct+=1
                    run=func_file.split("/")[-1].split("_")[3]
                    var_name="%s_%s_vol"%(task,run)
                    data_dict[subject][var_name]=vol

            if session == 'ses-2':
                data_dict[subject]["rl_file_ct"]=rl_ct
            data_dict[subject]["train_file_ct"]=train_ct
            data_dict[subject]["rest_file_ct"]=rest_ct
            # plot functionals
            #pool = Pool()
            #pool.map(plot_functionals, func_files)
            #pool.close()

        dataframe=pd.DataFrame(data_dict).T        
        dataframes.append(dataframe)
        #dataframe.to_excel(writer, sheet_name="%s_fmriprep"%session)
    #writer.save()
    #writer.close()
    return dataframes;


## Submit batch  
---

In [20]:
def submit_fmriprep_batch(job_file, x, submit_job=False):
    print('[INFO] batch file: %s'%job_file)
    batch_cmd='sbatch --array={}-{}%1 {}'.format(x,x,job_file)
    print('[INFO] batch command: {}'.format(batch_cmd))
    
    # submit batch job
    if submit_job==True: 
        sp.run(batch_cmd, shell=True)        
        print('[INFO] submitted job.')

In [21]:
bids_subs=[x.split("/")[-2] for x in 
           glob.glob(os.path.join(data_folder_path,"bids_trimmed/sub-*/ses-1"))]


In [24]:
bids_subs.remove('sub-161')
print(len(bids_subs))
print(bids_subs)


150
['sub-001', 'sub-002', 'sub-003', 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017', 'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-028', 'sub-030', 'sub-031', 'sub-032', 'sub-033', 'sub-034', 'sub-035', 'sub-036', 'sub-037', 'sub-038', 'sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-045', 'sub-046', 'sub-048', 'sub-050', 'sub-051', 'sub-052', 'sub-053', 'sub-054', 'sub-055', 'sub-056', 'sub-057', 'sub-058', 'sub-059', 'sub-060', 'sub-061', 'sub-062', 'sub-063', 'sub-064', 'sub-065', 'sub-066', 'sub-067', 'sub-068', 'sub-070', 'sub-071', 'sub-072', 'sub-073', 'sub-074', 'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-082', 'sub-083', 'sub-084', 'sub-085', 'sub-086', 'sub-087', 'sub-088', 'sub-089', 'sub-090', 'sub-091', 'sub-092', 'sub-093', 'sub-095', 'sub-096', 'sub-097', 'sub-

Set variables for batch command and submit job:  
`sbatch --array={start_id}-{finish_id}%{z} {job_file}`

In [25]:
# Set Parameters 
fmriprep_job_file='/projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job'
process_list=['sub-028', 'sub-030', 'sub-033', 'sub-035', 'sub-050', 'sub-052', 'sub-059',
             'sub-062', 'sub-065', 'sub-066', 'sub-068', 'sub-070', 'sub-072', 'sub-073',
             'sub-074', 'sub-075', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-082', 
             'sub-083', 'sub-084', 'sub-085', 'sub-086', 'sub-087', 'sub-088', 'sub-093',
             'sub-097', 'sub-108', 'sub-118', 'sub-128', 'sub-137', 'sub-168']

for sub_id in bids_subs:
    num_id=sub_id.split("-")[1].lstrip('0')
    
    submit_job=True # set to True when you want to run the file 
    submit_fmriprep_batch(fmriprep_job_file,num_id, submit_job)

[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=1-1%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=2-2%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=3-3%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=5-5%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/f

[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=39-39%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=40-40%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=41-41%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=42-42%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experi

[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=86-86%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=87-87%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=88-88%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=89-89%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experi

[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=132-132%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=133-133%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=134-134%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=135-135%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] submitted job.
[INFO] batch file: /projects/nibla

## Quality Report
---

In [98]:
quality_report()

[INFO] 151 session 1 fmriprep subjects found. 

[INFO] 1 session 1 bids subjects are not processed through fmriprep.

[INFO] ['sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017', 'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-028', 'sub-030', 'sub-031', 'sub-032', 'sub-033', 'sub-034', 'sub-035', 'sub-036', 'sub-037', 'sub-038', 'sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-045', 'sub-046', 'sub-048', 'sub-050', 'sub-051', 'sub-052', 'sub-053', 'sub-054', 'sub-055', 'sub-056', 'sub-057', 'sub-058', 'sub-059', 'sub-060', 'sub-061', 'sub-062', 'sub-063', 'sub-064', 'sub-065', 'sub-066', 'sub-067', 'sub-068', 'sub-070', 'sub-071', 'sub-072', 'sub-073', 'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-082', 'sub-083', 'sub-084', 'sub-085', 'sub-

In [99]:
fmriprep_dict={}


In [100]:
print("\n[INFO] fmriprep qc report")
print("[INFO] %s fmriprep session 1subject folders found."%len(fmriprep_ses1))
print("[INFO] %s fmriprep session 2 subject folders found."%len(fmriprep_ses2))



[INFO] fmriprep qc report
[INFO] 151 fmriprep session 1subject folders found.
[INFO] 129 fmriprep session 2 subject folders found.


In [101]:
fmriprep_ses2.sort()
print(fmriprep_ses2)

['sub-001', 'sub-002', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-013', 'sub-014', 'sub-015', 'sub-017', 'sub-019', 'sub-021', 'sub-022', 'sub-023', 'sub-027', 'sub-028', 'sub-030', 'sub-031', 'sub-032', 'sub-033', 'sub-034', 'sub-036', 'sub-037', 'sub-038', 'sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-045', 'sub-046', 'sub-048', 'sub-050', 'sub-051', 'sub-053', 'sub-054', 'sub-055', 'sub-057', 'sub-058', 'sub-061', 'sub-062', 'sub-063', 'sub-065', 'sub-066', 'sub-067', 'sub-068', 'sub-070', 'sub-071', 'sub-072', 'sub-073', 'sub-075', 'sub-078', 'sub-079', 'sub-080', 'sub-082', 'sub-083', 'sub-084', 'sub-085', 'sub-086', 'sub-088', 'sub-089', 'sub-090', 'sub-092', 'sub-093', 'sub-095', 'sub-096', 'sub-097', 'sub-098', 'sub-099', 'sub-100', 'sub-102', 'sub-103', 'sub-104', 'sub-107', 'sub-109', 'sub-111', 'sub-112', 'sub-114', 'sub-115', 'sub-116', 'sub-117', 'sub-118', 'sub-119', 'sub-120', 'sub-121', 'sub-123'

In [102]:
dataframes=build_fmriprep_report(write_files=False)

In [103]:
s2_df=dataframes[1]
s2_df.index.name = "patID"
s1_df=dataframes[0]
s1_df.index.name = "patID"

In [44]:
len([x for x in s2_df.index.values if x not in s2_drop_subjects])

131

In [105]:
s1_df.style.highlight_null('red')

,task-training_run-4_vol,task-training_run-3_vol,task-training_run-2_vol,task-training_run-1_vol,task-resting_vol,train_file_ct,rest_file_ct
patID,,,,,,,
sub-001,233,233,233,233,147,4,1
sub-002,233,233,233,233,147,4,1
sub-003,233,233,233,233,147,4,1
sub-004,233,233,233,233,147,4,1
sub-005,233,233,233,233,147,4,1
sub-006,233,233,233,233,147,4,1
sub-007,233,233,233,233,147,4,1
sub-008,233,233,233,233,147,4,1
sub-009,233,233,233,233,147,4,1


In [106]:
s2_df.style.highlight_null('red')

,task-training_run-4_vol,task-training_run-3_vol,task-training_run-2_vol,task-training_run-1_vol,task-rl_run-2_vol,task-rl_run-1_vol,task-resting_vol,rl_file_ct,train_file_ct,rest_file_ct
patID,,,,,,,,,,
sub-001,233,233,233,233,212,212,147,2,4,1
sub-002,233,233,233,233,212,212,147,2,4,1
sub-003,nan,nan,nan,nan,nan,nan,nan,0.000000,0.000000,0.000000
sub-004,233,233,233,233,212,212,147,2,4,1
sub-005,233,233,233,233,212,212,147,2,4,1
sub-006,233,233,233,233,212,212,147,2,4,1
sub-007,233,233,233,233,212,212,147,2,4,1
sub-008,233,233,233,233,212,212,147,2,4,1
sub-009,233,233,233,233,212,212,147,2,4,1


Write sheet to BBX data report excel file

In [107]:
path='/projects/niblab/experiments/bbx/data/quality_analysis/bbx_report.xlsx'


In [108]:
writer = pd.ExcelWriter(path, engine = 'openpyxl')
book = load_workbook(path)
writer.book = book
s1_df.to_excel(writer, sheet_name="ses-1_fmriprep")
s2_df.to_excel(writer, sheet_name="ses-2_fmriprep")
writer.save()
writer.close()

In [109]:
new_excel=pd.read_excel(path, sheet_name="ses-2_fmriprep")


In [110]:
new_excel.head()

,patID,task-training_run-4_vol,task-training_run-3_vol,task-training_run-2_vol,task-training_run-1_vol,task-rl_run-2_vol,task-rl_run-1_vol,task-resting_vol,rl_file_ct,train_file_ct,rest_file_ct
0,sub-001,233.0,233.0,233.0,233.0,212.0,212.0,147.0,2,4,1
1,sub-002,233.0,233.0,233.0,233.0,212.0,212.0,147.0,2,4,1
2,sub-003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,sub-004,233.0,233.0,233.0,233.0,212.0,212.0,147.0,2,4,1
4,sub-005,233.0,233.0,233.0,233.0,212.0,212.0,147.0,2,4,1


---

In [16]:
def display_fmriprep(sub_id=subject_list[0], sess_id=sessions[0]):
    print("[INFO] viewing report for %s"%sub_id)
    fmriprep_path=os.path.join(study_folder_path,'data/fmriprep')
    
    # anat figures
    anat_imgs=glob.glob(os.path.join(fmriprep_path, '%s/figures/*'%sub_id))
    anat_t1w=os.path.join(fmriprep_path, '%s/figures/%s_space-MNI152NLin2009cAsym_T1w.svg'%(sub_id,sub_id))
    try:
        display(SVG(filename=anat_t1w))
        #for svg in anat_imgs:
            #print("[INFO] viewing file %s"%svg)
            #display(SVG(filename=svg))
    except:
        print("[INFO] no anat figures found for %s"%sub_id)

# func figures
    print('\n[INFO] functional reports for %s'%sess_id)
    sess_svgs=glob.glob(os.path.join(fmriprep_path, '%s/%s/figures/*'%(sub_id,sess_id)))
    try:
        for svg in sess_svgs:
            if "flirt" in svg:
            #"brain_mask" in svg or "magnitude_bold" in svg or "rois" in svg or \
            #"flirt" in svg or "sdc" in svg or "fieldmap" in svg:
                print("[INFO] viewing file %s"%svg)
                display(SVG(filename=svg))
    except:
        print("[INFO] no anat figures found for %s"%sub_id)


In [17]:
# run widget that calls help functions above
w_fmriprep = widgets.Select(options=subject_list)
widgets.interactive(display_fmriprep, sub_id=subject_list, sess_id=sessions)

interactive(children=(Dropdown(description='sub_id', options=('sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub…

---